In [ ]:
%%configure -f
{"executorMemory": "12g", "executorCores": 4, "numExecutors":4}

In [ ]:
%%configure -f
{"conf": {"spark.jars": "/system/jar/simpleHTM.jar,/system/jar/jhealpix.jar"}}

In [ ]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf
import java.util.Calendar



In [ ]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)
    val healpixidUDF=udf(healpixid)
    
}

read object table parquet files into dataframe (objDF)

create new dataframe (newDF) with calculated htmid

In [ ]:
val datafile = "/user/hive/warehouse/object_test_newcols"

val newDF = spark.read.parquet(datafile)

//val objDF = spark.read.parquet(datafile)

//val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(objDF("ra"), objDF("decl")))

//val newDF = objDF.withColumn("htmid",HTMUtils.htmidUDF(objDF("ra"),objDF("decl")))
//newDF.printSchema()
//newDF.show(1)

In [ ]:
val authDF = spark.read.json("hdfs:///.config/creds.json")
lazy val m = authDF.first.getValuesMap[Any](authDF.schema.fieldNames)

lazy val username = (m("jdbc_username")).toString()
lazy val password = (m("jdbc_password")).toString()

write newDF to external table in SqlDataPool using Spark-SQL connector


In [ ]:

println("Use MSSQL connector to write to master SQL instance ")

val servername = "jdbc:sqlserver://master-0.master-svc"
val dbname = "LSST"
var url = servername + ";" + "databaseName=" + dbname + ";"


val datapool_table = "dp.Object_test2"

val datasource_name = "SqlDataPool"


//val batchsize = (1048576 * 10)
val batchsize = 1000000

val start = Calendar.getInstance().getTime()

try {
  newDF.write 
    .format("com.microsoft.sqlserver.jdbc.spark") 
    .mode("append") 
    .option("url", url) 
    .option("dbtable", datapool_table) 
    .option("user", username) 
    .option("password", password) 
    .option("dataPoolDataSource",datasource_name)
    .option("batchsize",batchsize)
    .save()
} catch {
    case e: Throwable => println("MSSQL Connector write failed: " + e)
}
val end = Calendar.getInstance().getTime()
println(start)
println(end)
//var duration = end - start
//println("duration:" + duration)
print("MSSQL Connector write(append) to data pool external table succeeded")